In [1]:
! pip install langchain_community langchainhub chromadb langchain langchain-chroma langchain-google-genai pypdf

In [4]:
os.environ['GOOGLE_API_KEY'] = ""

In [18]:
import bs4
import os
import time
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate

# Tạo template cho prompt

In [19]:
template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.

Question: {question}

Context: {context}

Answer:"""

prompt = ChatPromptTemplate.from_template(template)

# Crawl data

In [ ]:
### crawl từ web ###
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()


In [20]:
### crawl từ file pdf
DATA_PATH = "/content/data"

loader = PyPDFDirectoryLoader(DATA_PATH)
docs = loader.load()

In [22]:
print(docs)

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2026-02-02T21:20:41+07:00', 'author': 'Nguyen Thi Lan Anh  - dd23', 'moddate': '2026-02-02T21:20:41+07:00', 'source': '/content/data/Chinh_Sach_Doi_Tra_Bao_Hanh.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='CHÍNH SÁCH ĐỔI TRẢ VÀ BẢO HÀNH CỦA CỬA HÀNG SHATE SHOP \nNgày ban hành: 02/02/2026  \nÁp dụng cho: Tất cả các sản phẩm giày dép đư ợc bán t ại cửa hàng Shate Shop, bao \ngồm mua trực tiếp tại cửa hàng và mua hàng tr ực tuyến qua website hoặc các nền tảng \nliên kết.  \nMục đích: Shate Shop cam kết mang đến cho khách hàng những sản phẩm chất lượng \ncao và dịch vụ hậu mãi tốt nhất. Chính sách này nhằm bảo vệ quyền lợi của khách hàng, \nđồng thời quy định rõ ràng các điều kiện đổi trả và bảo hành để đảm bảo tính công bằng \nvà minh bạch. \nChính sách này tuân th ủ các quy định pháp luật Việt Nam liên quan đ ến bảo vệ quyền \nlợi n

# Chunking

In [21]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Indexing vào vector database

In [23]:
gemini_embeddings = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")

if os.path.exists("./chroma_db_pdf"):
    print("Đang nạp VectorDB từ ổ cứng...")
    vectorstore = Chroma(
        persist_directory="./chroma_db_pdf",
        embedding_function=gemini_embeddings
    )
else:
    print("Đang tạo mới VectorDB (Đang embedding)...")
    vectorstore = Chroma(
        embedding_function=gemini_embeddings,
        persist_directory="./chroma_db_pdf"
    )
    for i in range(0, len(splits), 2):
        batch = splits[i:i+2]
        vectorstore.add_documents(batch)
        print(f"Đã nạp xong {min(i+2, len(splits))}/{len(splits)} đoạn...")
        time.sleep(15)

retriever = vectorstore.as_retriever()

Đang nạp VectorDB từ ổ cứng...


# Retrieval

In [25]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt  # Sử dụng prompt vừa tạo ở bước 1
    | llm
    | StrOutputParser()
)

# Chạy thử
try:
    response = rag_chain.invoke("Cách bảo quản giày thể thao?")
    print("\n--- KẾT QUẢ ---")
    print(response)
except Exception as e:
    print(f"\nVẫn gặp lỗi: {e}")


--- KẾT QUẢ ---
Để bảo quản giày thể thao, bạn nên tháo dây và lót giày để vệ sinh riêng, lau đế bằng khăn ẩm và xà phòng nhẹ, còn thân giày thì vệ sinh tùy theo chất liệu. Tránh phơi nắng trực tiếp, hãy để giày nơi khô ráo, thoáng mát và nhét giấy báo hoặc shoe tree ngay sau khi tháo để giữ dáng. Ngoài ra, bạn có thể dùng túi hút ẩm hoặc baking soda để khử mùi hôi và xịt chống thấm nước định kỳ cho giày.
